In [ ]:
import tensorflow as tf
tf.__version__
#!pip uninstall tensorflow
#!pip install tensorflow
#%tensorflow_version 1.x

import keras
print(keras.__version__)
!pip install keras==2.14.0


In [ ]:
# 2D TRAIN AND VAL Data generator

from google.colab import drive
import os
import random
import numpy as np
from PIL import Image

from PIL import Image
import numpy as np

def load_image(file_path):
    image = Image.open(file_path)

    # Crop in square form
    image = image.crop((0, 0, 256, 256))

    # Convert image to NumPy array
    image_array = np.array(image)

    return image_array

def train_val_data_generator(blur_folder, crisp_folder, batch_size):

    validation_split = 0.05
    num_validation = int(validation_split * batch_size) + 1
    num_train = batch_size - num_validation

    # Infinite loop to yield batches during training
    while True: # Random selection of images in multiple folders

        # Initialize lists to store mini-stacks and corresponding crisp images
        X_batch = []
        Y_batch = []
        count = 0;
        print(f"Generating 2D pairs...")
        # Iterate over the indices in the batch
        while count != batch_size:
            count+=1

            folders = ["480490_541480", "500480_533880", "500484_508050", "533470_523780"]

            # Define the mapping of folders to num_images
            folder_to_num_images = {
                "480490_541480": 2408,
                "500480_533880": 2888,
                "500484_508050": 2888,
                "533470_523780": 2862
            }
            random_folder = random.choice(folders)

            num_images = folder_to_num_images.get(random_folder, 0)

            random_ind = random.randint(200, num_images-200)

            crisp_image = load_image(os.path.join(crisp_folder, f"{random_folder}", f"{random_folder}_{random_ind:04d}.tif"))
            blur_image = load_image(os.path.join(blur_folder, f"{random_folder}", f"{random_ind:05d}.tif"))
            X_batch.append(blur_image)
            Y_batch.append(crisp_image)

            #print(f": {count}/{batch_size}, {random_folder}_{random_ind:04d}.tif")

        X_train, Y_train = X_batch[:num_train], Y_batch[:num_train]
        X_val, Y_val = X_batch[num_train:], Y_batch[num_train:]


        print(f"Shape of X_train: {X_train[0].shape}")
        print(f"Shape of Y_train: {Y_train[0].shape}")

        print(f"Image pairs prepared. Train/Val = {len(X_train)}/{len(X_val)}")

        yield (X_train, Y_train), (X_val, Y_val)


drive.mount('/content/gdrive')
train_blur_folder = "gdrive/My Drive/Noisy_data/"
train_crisp_folder = "gdrive/My Drive/Crisp_data/"


batch_size = 5

train_val_data = train_val_data_generator(train_blur_folder, train_crisp_folder, batch_size)

(X_train, Y_train), (X_val, Y_val) = next(train_val_data)

X_min_value = np.min(X_train[0])
X_max_value = np.max(X_train[0])
Y_min_value = np.min(Y_train[0])
Y_max_value = np.max(Y_train[0])

print(f"X_train min/max Value: {X_min_value}/{X_max_value}")
print(f"Y_train min/max Value: {Y_min_value}/{Y_max_value}")




In [ ]:
# UNet USING TRAIN VAL GENERATOR

import tensorflow as tf
from tensorflow.keras import layers, models
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from tensorflow.keras.models import load_model
from tensorflow.keras.models import save_model
from google.colab import drive
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanAbsoluteError
from tensorflow.keras.optimizers import Adam

import os
import pandas as pd
import numpy as np

import PIL
import PIL.Image
import pathlib
import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

import io
import random

# Parameters
conv_filt       = 32
kernel_size     = 3
activation      = "relu"
padding         = "same"
pool_size       = (2, 2)

# TRAIN
optimizer       = "Adam"
learning_rate   = 1e-4
validation_split = 0.1
train_eval_ratio = 0.9
input_shape = [256,256,1]


batch_size = 16

################################################################################

# Simple 2D U-Net
def get_model_unet(input_shape = input_shape,
                             conv_filt=32,
                             kernel_size=3,
                             activation="relu",
                             padding="same",
                             pool_size=pool_size):
    conv_args = {"activation": activation,
                 "padding": padding,
                 "kernel_size": kernel_size}
    input_ = Input(shape=input_shape)
    conv1 = Conv2D(filters=conv_filt, **conv_args)(input_)
    conv2 = Conv2D(filters=conv_filt, **conv_args)(conv1)
    pool1 = MaxPool2D(pool_size=pool_size)(conv2)
    #
    conv3 = Conv2D(filters=2*conv_filt, **conv_args)(pool1)
    conv4 = Conv2D(filters=2*conv_filt, **conv_args)(conv3)
    pool2 = MaxPool2D(pool_size=pool_size)(conv4)
    #
    conv5 = Conv2D(filters=4*conv_filt, **conv_args)(pool2)
    conv6 = Conv2D(filters=2*conv_filt, **conv_args)(conv5)
    up1 = UpSampling2D(size=(2,2))(conv6)
    #
    conc1 = Concatenate()([conv4, up1])
    #
    conv7 = Conv2D(filters=2*conv_filt, **conv_args)(conc1)
    conv8 = Conv2D(filters=conv_filt, **conv_args)(conv7)
    up2 = UpSampling2D(size=(2,2))(conv8)
    #
    conc2 = Concatenate()([conv2, up2])
    #
    conv9 = Conv2D(filters=conv_filt, **conv_args)(conc2)
    conv10 = Conv2D(filters=conv_filt, **conv_args)(conv9)
    #
    output = Conv2D(filters=1, kernel_size=1, activation=None)(conv10)
    #
    model = keras.Model(inputs=[input_], outputs=[output])
    return model

# Define model
model   = get_model_unet(
    input_shape=input_shape,
    conv_filt=conv_filt,
    kernel_size=kernel_size,
    activation=activation,
    padding=padding,
    pool_size=pool_size)

loss_fn = keras.losses.MeanSquaredError()
optimizer = keras.optimizers.Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer,
          loss=loss_fn,
          metrics=["mse"])

model.summary()


################################################################################


# Train the model using the data generator
train_val_data = train_val_data_generator(train_blur_folder, train_crisp_folder, batch_size)

psnr_history = []
ssim_history = []
pcc_history = []

# Training loop
epochs = 1000  # Adjust as needed
for epoch in range(epochs):
    (X_train, Y_train), (X_val, Y_val) = next(train_val_data)

    # Train the model
    history = model.fit(np.array(X_train), np.array(Y_train), epochs=1, validation_data=(np.array(X_val), np.array(Y_val)))

    # Evaluate the model on the validation set
    val_preds = model.predict(np.array(X_val))

    val_preds_min_value = np.min(val_preds[0])
    val_preds_max_value = np.max(val_preds[0])
    gt_min_value = np.min(Y_val[0])
    gt_max_value = np.max(Y_val[0])

    print(f"val_preds Min/Max Value: {val_preds_min_value} / {val_preds_max_value}")
    print(f"ground-truth Min/Max Value: {gt_min_value} / {gt_max_value}")

    # Calculate and print PSNR and SSIM for the test set
    psnr_val_list = [psnr(np.squeeze(Y_val[i]), np.squeeze(val_preds[i]), data_range=65535.0) for i in range(len(Y_val))]
    ssim_val_list = [ssim(np.squeeze(Y_val[i]), np.squeeze(val_preds[i]), data_range=65535.0) for i in range(len(Y_val))]
    pcc_val_list = [pearsonr(np.squeeze(Y_val[i]).ravel(), np.squeeze(val_preds[i]).ravel())[0] for i in range(len(Y_val))]

    # Report the average PSNR, SSIM, and PCC for the test set
    average_psnr_val = np.mean(psnr_val_list)
    average_ssim_val = np.mean(ssim_val_list)
    average_pcc_val = np.mean(pcc_val_list)

    # Store PSNR, SSIM, and PCC values in history lists
    psnr_history.append(average_psnr_val)
    ssim_history.append(average_ssim_val)
    pcc_history.append(average_pcc_val)

    # Plotting real-time PSNR and SSIM
    plt.figure(figsize=(12, 4))

    # Plot PSNR
    plt.subplot(1, 3, 1)
    plt.plot(psnr_history, label='PSNR')
    plt.xlabel('Epoch')
    plt.ylabel('PSNR')
    plt.title('Real-time PSNR during Validation')
    plt.legend()

    # Plot SSIM
    plt.subplot(1, 3, 2)
    plt.plot(ssim_history, label='SSIM', color='orange')
    plt.xlabel('Epoch')
    plt.ylabel('SSIM')
    plt.title('Real-time SSIM during Validation')
    plt.legend()

     # Plot PCC
    plt.subplot(1, 3, 3)
    plt.plot(pcc_history, label='PCC', color='green')
    plt.xlabel('Epoch')
    plt.ylabel('PCC')
    plt.title('Real-time PCC during Validation')
    plt.legend()

    plt.tight_layout()
    plt.show()

    print(f'Epoch {epoch + 1}/{epochs} - PSNR: {average_psnr_val}, SSIM: {average_ssim_val}')

    # Display the original (noisy) image
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    plt.imshow(np.squeeze(X_val[0]) * 1.0, cmap='gray', vmin=0, vmax=500)
    plt.title('Original (Noisy) Image')
    plt.axis('off')

    # Display the predicted (crisp) image
    plt.subplot(1, 3, 2)
    plt.imshow(np.squeeze(val_preds[0]) * 1.0, cmap='gray', vmin=0, vmax=500)
    plt.title('Predicted (Crisp) Image')
    plt.axis('off')

    # Display the ground truth image
    plt.subplot(1, 3, 3)
    plt.imshow(np.squeeze(Y_val[0]) * 1.0, cmap='gray', vmin=0, vmax=500)
    plt.title('Ground Truth Image')
    plt.axis('off')

    plt.tight_layout()
    plt.show()

from datetime import datetime

# Get the current date and time
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
# Define the model file name with the current time
model_file_name = f'model_ds_{current_time}.keras'
model_save_path = f'gdrive/My Drive/{model_file_name}'

# Save the model to Google Drive
save_model(model, model_save_path)
print(f'Model saved to {model_save_path}')


In [ ]:
# 2D TEST Data generator (RANDOM NOISE)

from google.colab import drive
import os
import random
import numpy as np
from PIL import Image

def load_image(file_path):
    image = Image.open(file_path)

    # Crop in square form
    image = image.crop((244, 144, 500, 400))

    # Convert image to NumPy array
    image_array = np.array(image) / 1 #65535.0

    return image_array

def test_data_generator(blur_folder, crisp_folder, batch_size):

    validation_split = 0.1
    num_validation = int(validation_split * batch_size) + 1
    num_train = batch_size - num_validation

    # Infinite loop to yield batches during training
    while True: # Random selection of images in multiple folders

        # Initialize lists to store mini-stacks and corresponding crisp images
        X_test = []
        Y_test = []
        count = 0;
        print(f"Generating TEST 2D pairs...")
        # Iterate over the indices in the batch
        while count != batch_size:
            count+=1

            random_ind = random.randint(1000, 2000)

            print(f": {count}/{batch_size}, {random_ind:05d}.tif")

            crisp_image = load_image(os.path.join(crisp_folder, f"500480_482220_{random_ind:04d}.tif"))


            std_dev = random.randint(30, 40)

            # Generate Gaussian noise with the same shape as the image
            noise = np.random.normal(loc=0, scale=std_dev, size=crisp_image.shape)

            # Add the noise to the image
            noisy_image = crisp_image + noise

            # Clip the pixel values to the valid range [0, 65535.0]
            noisy_image = np.clip(noisy_image, 0, 65535.0)

            # Assuming 16-bit images, changing the scale to [0,1]
            X_test.append(noisy_image)
            Y_test.append(crisp_image)


        print(f"Shape of X_train: {X_test[0].shape}")
        print(f"Shape of Y_train: {Y_test[0].shape}")
        print(f"TEST image pairs prepared. Testlength = {len(X_test)}")

        yield X_test, Y_test


drive.mount('/content/gdrive')
test_blur_folder = "gdrive/My Drive/Noisy_data/500480_482220/"
test_crisp_folder = "gdrive/My Drive/Crisp_data/500480_482220/"


batch_size = 5

test_data = test_data_generator(test_blur_folder, test_crisp_folder, batch_size)

X_test, Y_test = next(test_data)

X_min_value = np.min(X_test[0])
X_max_value = np.max(X_test[0])
Y_min_value = np.min(Y_test[0])
Y_max_value = np.max(Y_test[0])

print(f"X_test min/max Value: {X_min_value}/{X_max_value}")
print(f"Y_test min/max Value: {Y_min_value}/{Y_max_value}")

In [ ]:
# Display the image of test result

import tensorflow as tf
from tensorflow.keras import layers, models
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from tensorflow.keras.models import load_model
from tensorflow.keras.models import save_model
from google.colab import drive
import matplotlib.pyplot as plt
import numpy as np
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from scipy.stats import pearsonr
import cv2
import matplotlib.pyplot as plt
from skimage.restoration import denoise_tv_bregman


# Load the saved model from Google Drive
model_save_path = f'gdrive/My Drive/model_ds_20231203_182307.keras'
loaded_model = load_model(model_save_path)

batch_size = 20

test_data = test_data_generator(test_blur_folder, test_crisp_folder, batch_size)
X_test, Y_test = next(test_data)

# Use the loaded model to predict the crisp image
test_pred = loaded_model.predict(np.array(X_test))

#################################################################################

# Define functions for metrics calculation
def calculate_metrics(original, denoised):
    psnr_value = psnr(np.squeeze(original), np.squeeze(denoised), data_range=65535.0)
    ssim_value = ssim(np.squeeze(original), np.squeeze(denoised), data_range=65535.0)
    pcc_value, _ = pearsonr(original.flatten(), denoised.flatten())
    return psnr_value, ssim_value, pcc_value

# Function to display images and calculate metrics
def display_and_calculate_metrics(batch_index, original, denoised_tv, denoised_cv2, predicted, ground_truth):
    # Display images
    plt.figure(figsize=(30, 10))

    plt.subplot(1, 5, 1)
    plt.imshow(original, cmap='gray', vmin=0, vmax=500)
    plt.title('Original')
    plt.axis('off')

    plt.subplot(1, 5, 2)
    plt.imshow(denoised_tv, cmap='gray', vmin=0, vmax=500)
    plt.title('Denoised (TV)')
    plt.axis('off')

    plt.subplot(1, 5, 3)
    plt.imshow(denoised_cv2, cmap='gray', vmin=0, vmax=500)
    plt.title('Denoised (cv2)')
    plt.axis('off')

    plt.subplot(1, 5, 4)
    plt.imshow(predicted, cmap='gray', vmin=0, vmax=500)
    plt.title('Predicted')
    plt.axis('off')

    plt.subplot(1, 5, 5)
    plt.imshow(ground_truth, cmap='gray', vmin=0, vmax=500)
    plt.title('Ground Truth')
    plt.axis('off')

    # Calculate and print metrics
    psnr_tv, ssim_tv, pcc_tv = calculate_metrics(ground_truth, denoised_tv)
    psnr_cv2, ssim_cv2, pcc_cv2 = calculate_metrics(ground_truth, denoised_cv2)
    psnr_pred, ssim_pred, pcc_pred = calculate_metrics(ground_truth, predicted)


    '''
    print(f"Batch {batch_index + 1} Metrics:")
    print(f"PSNR (TV): {psnr_tv:.4f}, SSIM (TV): {ssim_tv:.4f}, PCC (TV): {pcc_tv:.4f}")
    print(f"PSNR (cv2): {psnr_cv2:.4f}, SSIM (cv2): {ssim_cv2:.4f}, PCC (cv2): {pcc_cv2:.4f}")
    print(f"PSNR (Prediction): {psnr_pred:.4f}, SSIM (Prediction): {ssim_pred:.4f}, PCC (Prediction): {pcc_pred:.4f}")
    print()
    '''

# Lists to store metric values for each image
psnr_tv_list, ssim_tv_list, pcc_tv_list = [], [], []
psnr_cv2_list, ssim_cv2_list, pcc_cv2_list = [], [], []
psnr_pred_list, ssim_pred_list, pcc_pred_list = [], [], []


# Iterate through batches
num_batches = len(X_test)
average_psnr_tv, average_ssim_tv, average_pcc_tv = 0, 0, 0
average_psnr_cv2, average_ssim_cv2, average_pcc_cv2 = 0, 0, 0
average_psnr_pred, average_ssim_pred, average_pcc_pred = 0, 0, 0

for batch_index in range(num_batches):
    original_image = X_test[batch_index]
    ground_truth_image = Y_test[batch_index]
    predicted_image = test_pred[batch_index]



    # Denoise using denoise_tv_bregman
    denoised_tv = denoise_tv_bregman(original_image, weight=0.1)

    # Convert the images to 8-bit unsigned
    bit8_original_image = cv2.convertScaleAbs(original_image)

    # Denoise using cv2.fastNlMeansDenoising
    denoised_cv2 = cv2.fastNlMeansDenoising(bit8_original_image, None, h=10, templateWindowSize=7, searchWindowSize=21)

    # Display images and calculate metrics
    display_and_calculate_metrics(batch_index, original_image, denoised_tv, denoised_cv2, predicted_image, ground_truth_image)

    # Update running totals for average metrics
    psnr_tv, ssim_tv, pcc_tv = calculate_metrics(ground_truth_image, denoised_tv)
    psnr_cv2, ssim_cv2, pcc_cv2 = calculate_metrics(ground_truth_image, denoised_cv2)
    psnr_pred, ssim_pred, pcc_pred = calculate_metrics(ground_truth_image, predicted_image)

    # Append metrics to lists
    psnr_tv_list.append(psnr_tv)
    ssim_tv_list.append(ssim_tv)
    pcc_tv_list.append(pcc_tv)

    psnr_cv2_list.append(psnr_cv2)
    ssim_cv2_list.append(ssim_cv2)
    pcc_cv2_list.append(pcc_cv2)

    psnr_pred_list.append(psnr_pred)
    ssim_pred_list.append(ssim_pred)
    pcc_pred_list.append(pcc_pred)

    average_psnr_tv += psnr_tv
    average_ssim_tv += ssim_tv
    average_pcc_tv += pcc_tv

    average_psnr_cv2 += psnr_cv2
    average_ssim_cv2 += ssim_cv2
    average_pcc_cv2 += pcc_cv2

    average_psnr_pred += psnr_pred
    average_ssim_pred += ssim_pred
    average_pcc_pred += pcc_pred

# Calculate average metrics across all batches
average_psnr_tv /= num_batches
average_ssim_tv /= num_batches
average_pcc_tv /= num_batches

average_psnr_cv2 /= num_batches
average_ssim_cv2 /= num_batches
average_pcc_cv2 /= num_batches

average_psnr_pred /= num_batches
average_ssim_pred /= num_batches
average_pcc_pred /= num_batches

# Print average metrics
print("Average Metrics Across All Batches:")
print(f"PSNR (TV): {average_psnr_tv:.4f}, SSIM (TV): {average_ssim_tv:.4f}, PCC (TV): {average_pcc_tv:.4f}")
print(f"PSNR (cv2): {average_psnr_cv2:.4f}, SSIM (cv2): {average_ssim_cv2:.4f}, PCC (cv2): {average_pcc_cv2:.4f}")
print(f"PSNR (Prediction): {average_psnr_pred:.4f}, SSIM (Prediction): {average_ssim_pred:.4f}, PCC (Prediction): {average_pcc_pred:.4f}")

# Plot graphs
image_numbers = np.arange(1, num_batches + 1)

plt.figure(figsize=(15, 5))

# PSNR
plt.subplot(1, 3, 1)
plt.plot(image_numbers, psnr_tv_list, marker='o', label='TV')
plt.plot(image_numbers, psnr_cv2_list, marker='o', label='cv2')
plt.plot(image_numbers, psnr_pred_list, marker='o', label='Prediction')
plt.xlabel('Image Number')
plt.ylabel('PSNR')
plt.legend()
plt.title('PSNR Comparison')

# SSIM
plt.subplot(1, 3, 2)
plt.plot(image_numbers, ssim_tv_list, marker='o', label='TV')
plt.plot(image_numbers, ssim_cv2_list, marker='o', label='cv2')
plt.plot(image_numbers, ssim_pred_list, marker='o', label='Prediction')
plt.xlabel('Image Number')
plt.ylabel('SSIM')
plt.legend()
plt.title('SSIM Comparison')

# PCC
plt.subplot(1, 3, 3)
plt.plot(image_numbers, pcc_tv_list, marker='o', label='TV')
plt.plot(image_numbers, pcc_cv2_list, marker='o', label='cv2')
plt.plot(image_numbers, pcc_pred_list, marker='o', label='Prediction')
plt.xlabel('Image Number')
plt.ylabel('PCC')
plt.legend()
plt.title('PCC Comparison')

plt.tight_layout()
plt.show()

